In [2]:
%load_ext autoreload
%autoreload 2
from covid_constants_and_util import *
import helper_methods_for_aggregate_data_analysis as helper
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
import datetime
import scipy
import time

JUST_TESTING = False

Setting numpy cores to 1
Running code on Deep; at Stanford=False


KeyboardInterrupt: 

In [ ]:
COLS_FROM_CORE_PLACES = ['safegraph_place_id', 'parent_safegraph_place_id', 'location_name', 'safegraph_brand_ids', 
                         'brands', 'top_category', 'sub_category', 'naics_code', 'latitude', 'longitude', 'street_address',
                         'city', 'region', 'postal_code', 'open_hours']
COLS_FROM_CORE_PLACES_FOOTPRINT = ['safegraph_place_id', 'polygon_class', 'area_square_feet']
COLS_FROM_WEEKLY_PATTERNS = ['safegraph_place_id', 'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs', 
                             'visitor_country_of_origin', 'distance_from_home', 'median_dwell', 'bucketed_dwell_times']

In [ ]:
core_poi_df = helper.load_core_places_data(COLS_FROM_CORE_PLACES)

In [ ]:
footprint_df = helper.load_core_places_footprint_data(COLS_FROM_CORE_PLACES_FOOTPRINT)
ids_in_footprint_but_not_core = set(footprint_df.index) - set(core_poi_df.index)
print("Warning: %i/%i (%.1f%%) POIs in footprint but not core data; dropping these" % 
      (len(ids_in_footprint_but_not_core), len(footprint_df), 100. * len(ids_in_footprint_but_not_core) / len(footprint_df)))
core_poi_df = pd.merge(core_poi_df, footprint_df, how='left', left_index=True, right_index=True, validate='one_to_one')
print("Missing data for footprint columns")
print(pd.isnull(core_poi_df[footprint_df.columns]).mean())

# Combine Core/Footprint/Weekly data and write out to MSA files

In this section, we process the raw SafeGraph Weekly Patterns data and divide the processing output into separate csv files per metropolitan statistical area (MSA). In processing the data, we:
- Expand the visits_by_each_hour column into one column per hour.
- Correct the visit spikes at the GMT midnight boundary (this is a known error with hourly SafeGraph data).
- Drop parent POIs so we don't double-count visits.
- Add an MSA column based on the POI's census block group (if that information is provided).

In [ ]:
# first and last week of the latest Weekly data to process
start_date = datetime.datetime(2021, 2, 3)   # Should be the data release date
end_date = datetime.datetime(2021, 2, 9)
print('Time period: %s to %s (%s)' % (start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), end_date - start_date))

In [ ]:
base = core_poi_df.copy()
while start_date <= end_date:
    week_string = start_date.strftime('%Y-%m-%d')
    print('===== WEEK STRING: %s =====' % week_string)
    weekly_df = helper.load_weekly_patterns_v2_data(week_string, COLS_FROM_WEEKLY_PATTERNS, expand_hourly_visits=True)
    weekly_df = weekly_df.rename(columns={k:f'{week_string}.{k}' for k in weekly_df.columns if not(k.startswith('hourly_visits_'))})
    ids_in_weekly_but_not_core = set(weekly_df.index) - set(base.index)
    print("Warning: %i/%i (%.1f%%) POIs in weekly but not core data; dropping these" % 
          (len(ids_in_weekly_but_not_core), len(weekly_df), 100. * len(ids_in_weekly_but_not_core) / len(weekly_df)))
    # left merge means we are only keeping the POIs that we have core info for
    base = pd.merge(base, weekly_df, how='left', left_index=True, right_index=True, validate='one_to_one')
    print("Missing data for weekly columns")
    print(pd.isnull(base[weekly_df.columns]).mean())
    start_date = start_date + datetime.timedelta(days=7)

In [ ]:
def get_single_poi_cbg_col(base):
    poi_cbg_cols = sorted([col for col in base.columns if col.endswith('poi_cbg')])  # put into temporal order
    current_data = base[poi_cbg_cols[0]].copy()
    print('Processed col %s -> num POIs currently missing CBG: %d' % (poi_cbg_cols[0], np.sum(pd.isnull(current_data))))
    for col_name in poi_cbg_cols[1:]:
        new_data = base[col_name]
        non_null = ~pd.isnull(new_data)
        current_data.loc[non_null] = new_data.loc[non_null]
        print('Processed col %s -> num POIs currently missing CBG: %d' % (col_name, np.sum(pd.isnull(current_data))))
    return current_data

base['single_poi_cbg'] = get_single_poi_cbg_col(base)

In [ ]:
base

In [ ]:
pd.set_option('display.max_columns', None)
base.head(5)


In [ ]:
# extract population from ACS geodatabase
# acs5_gdf = gpd.read_file(PATH_TO_ACS_5YR_DATA_GEODATABASE, layer=r'')

In [ ]:
def map_poi_cbg_to_msa_info(x, msa_info):
    if pd.isnull(x):
        return 'Missing POI CBG'
    x_as_int = int(x)
    if x_as_int in msa_info:
        return msa_info[x_as_int]
    return 'Missing ACS data'

acs_d = helper.load_and_reconcile_multiple_acs_data()
cbg2metro_micro = dict(zip(acs_d.census_block_group, acs_d['Metropolitan/Micropolitan Statistical Area']))
cbg2cbsa = dict(zip(acs_d.census_block_group, acs_d['CBSA Title']))
base['poi_metro_micro'] = base['single_poi_cbg'].map(lambda x:map_poi_cbg_to_msa_info(x, cbg2metro_micro))
base['poi_cbsa'] = base['single_poi_cbg'].map(lambda x:map_poi_cbg_to_msa_info(x, cbg2cbsa))
base['poi_cbsa'].value_counts()

In [ ]:
just_in_msas = base[base.poi_metro_micro == 'Metropolitan Statistical Area']
print("%i/%i POIs are in MSAs (%i MSAs total)" % (len(just_in_msas), 
                                                  len(base), 
                                                  len(just_in_msas['poi_cbsa'].unique())))

In [ ]:
OUT_DIR = os.path.join(NEW_STRATIFIED_BY_AREA_DIR, '20210125_20210201')
assert os.path.isdir(OUT_DIR)
print(OUT_DIR)

### Write to file: if we're starting a new chunk in stratified_by_metro_area

In [ ]:
grouped_by_msa = just_in_msas.groupby('poi_cbsa')
total_written_out = 0
for msa_name, small_d in grouped_by_msa:
    small_d = small_d.copy().sample(frac=1) # make sure rows in random order. 
    name_without_spaces = re.sub('[^0-9a-zA-Z]+', '_', msa_name)
    filename = os.path.join(OUT_DIR, '%s.csv' % name_without_spaces)
    small_d.to_csv(filename)
    print("Wrote out dataframe with %i POIs to %s.csv" % (len(small_d), name_without_spaces))
    total_written_out += 1
print("Total written out: %i" % total_written_out)

### Write to file: if we're concatenating with existing chunk in stratified_by_metro_area

In [ ]:
grouped_by_msa = just_in_msas.groupby('poi_cbsa')
total_written_out = 0
prev_out_dir = os.path.join(NEW_STRATIFIED_BY_AREA_DIR, '20201026_20201228')
assert prev_out_dir != OUT_DIR
for msa_name, small_d in grouped_by_msa:
    ts = time.time()
    name_without_spaces = re.sub('[^0-9a-zA-Z]+', '_', msa_name)
    filename = os.path.join(OUT_DIR, '%s.csv' % name_without_spaces)
    prev_filename = os.path.join(prev_out_dir, '%s.csv' % name_without_spaces)
    prev_d = pd.read_csv(prev_filename)
    prev_d = prev_d.set_index('safegraph_place_id')
    new_cols = [col for col in small_d.columns if col not in prev_d]
    merged_d = prev_d.merge(small_d[new_cols], how='outer', left_index=True, right_index=True)
    merged_d.to_csv(filename)
    print("Wrote out dataframe with %i POIs to %s.csv [time=%.3fs]" % 
          (len(merged_d), name_without_spaces, time.time()-ts))
    total_written_out += 1
print("Total written out: %i" % total_written_out)

## Test written out MSA files

In [ ]:
msa_name = 'Washington_Arlington_Alexandria_DC_VA_MD_WV'
time_period_strings = ['20200302_20200608', '20200615_20200817', '20200824_20201019', '20201026_20210118', '20210125_20210201']
dfs = []
for ts in time_period_strings:
    df = helper.load_dataframe_for_individual_msa(msa_name, version='v2', time_period=ts)
    dfs.append(df)

In [ ]:
all_hours = []
visits_per_hour = []
for df in dfs:
    hourly_visit_cols = [col for col in df.columns if col.startswith('hourly_visits')]
    start_hour_string = hourly_visit_cols[0].split('_')[-1]
    start_year, start_month, start_day, start_hour = start_hour_string.split('.')
    start_hour = datetime.datetime(int(start_year), int(start_month), int(start_day), int(start_hour))
    end_hour_string = hourly_visit_cols[-1].split('_')[-1]
    end_year, end_month, end_day, end_hour = end_hour_string.split('.')
    end_hour = datetime.datetime(int(end_year), int(end_month), int(end_day), int(end_hour))
    if len(all_hours) > 0:
        assert start_hour == (all_hours[-1] + datetime.timedelta(hours=1))
    hours_in_range = helper.list_hours_in_range(start_hour, end_hour)
    all_hours.extend(hours_in_range)
    mat = df[hourly_visit_cols].values
    num_nonnan_per_hour = np.sum(~np.isnan(mat), axis=0)
    num_avg_visits_per_hour = np.nansum(mat, axis=0) / num_nonnan_per_hour
    visits_per_hour.extend(num_avg_visits_per_hour)
    
    print('%s - %s (%d hours) -> num non-nan POIs: avg=%.2f, std=%.2f; avg visits per POI: avg=%.2f, std=%.2f' % 
      (start_hour.strftime('%Y/%m/%d, %H'), end_hour.strftime('%Y/%m/%d, %H'), len(hours_in_range),
       np.mean(num_nonnan_per_hour), np.std(num_nonnan_per_hour), np.mean(num_avg_visits_per_hour), np.std(num_avg_visits_per_hour)))

In [ ]:
from matplotlib import ticker as tick

fig, ax = plt.subplots(figsize=(15, 5))
x = all_hours
y = apply_smoothing(visits_per_hour, before=24, after=24)
ax.plot_date(x, y, linestyle='-', marker='.')
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.set_xlabel('Date (every 3 Sundays)', fontsize=16)
ax.set_ylabel('Average num visits per POI\n(with 2-day smoothing)', fontsize=16)
ax.set_title('%s' % MSAS_TO_PRETTY_NAMES[msa_name], fontsize=18)
ax.tick_params(labelsize=14)
ax.grid(alpha=0.2)
plt.show()

# Post-processing on MSA files: create time-aggregated

In this section, we aggregate data over many weeks to provide time-aggregated estimates for certain POI attributes. 
- During network inference (see generate_ipf.py), we use time-aggregated estimates of each POI's visitors' home CBG distribution.
- In our model (see model_experiments.py), we use time-aggregated estimates for each POI's dwell time.

Since we use time-aggregated estimates, we do not update these values every time we download new SafeGraph data. As of February 2021, we are using the weeks from 2019/12/30 to 2020/10/19 as the basis for time-aggregated information.

In [ ]:
pop_df = helper.load_dataframe_to_correct_for_population_size(version='v2', verbose=False)
pop_df.head()

In [ ]:
# constant info: core, area, aggregated dwell time, aggregated home cbgs
def make_time_aggregated_file_for_msa(msa_name, core_poi_df, pop_df, time_period_strings, dfs=None):
    prefix = '%s_%s' % (time_period_strings[0].split('_')[0], time_period_strings[-1].split('_')[1])
    print('Prefix:', prefix)
    pois_in_msa = set()
    if dfs is None:
        dfs = []
        for ts in time_period_strings:
            df = helper.load_dataframe_for_individual_msa(msa_name, version='v2', time_period=ts)
            pois_in_msa = pois_in_msa.union(df.index)
            dfs.append(df)
    else:
        copied_dfs = []
        for df in dfs:
            copied_dfs.append(df.copy())  # don't want to modify by accident
            pois_in_msa = pois_in_msa.union(df.index)
        dfs = copied_dfs
    
    pois_in_msa = list(sorted(pois_in_msa))
    base = core_poi_df.loc[pois_in_msa].copy()
    parent_ids = set(core_poi_df.parent_safegraph_place_id)
    is_parent = base.index.isin(parent_ids)
    parents = base[is_parent]
    nonparents = base[~is_parent]
    print('In MSA: %d parent POIs; %d non-parent POIs' % (len(parents), len(nonparents)))
    base = nonparents    
    base['poi_cbg'] = np.nan
    for df in dfs:
        visitor_home_cbg_cols = sorted([col for col in df.columns if col.endswith('visitor_home_cbgs')])
        median_dwell_cols = sorted([col for col in df.columns if col.endswith('median_dwell')])
        cols_to_keep = ['single_poi_cbg'] + visitor_home_cbg_cols + median_dwell_cols
        base = pd.merge(base, df[cols_to_keep], how='left', left_index=True, right_index=True, validate='one_to_one')
        poi_cbg_is_nan = pd.isnull(base['poi_cbg'])  # update poi_cbg data that is still null
        base['poi_cbg'][poi_cbg_is_nan] = base['single_poi_cbg'][poi_cbg_is_nan]  # each df has a single_poi_cbg column
        base = base.drop(columns=['single_poi_cbg'])
        
    visitor_home_cbg_cols = [col for col in base.columns if col.endswith('visitor_home_cbgs')]
    print('Found %d visitor home CBG columns' % len(visitor_home_cbg_cols))
    is_null_mat = pd.isnull(base[visitor_home_cbg_cols])
    base['num_weeks_with_visitor_home_cbgs'] = np.sum(~is_null_mat, axis=1)
    periods_to_include = [c[:10] for c in visitor_home_cbg_cols]  # only keep date string
    base = helper.aggregate_visitor_home_cbgs_over_months(base, population_df=pop_df, 
                                                cutoff_year=None, periods_to_include=periods_to_include)
    base = base.rename(columns={k:'%s_%s' % (prefix, k) for k in ['aggregated_cbg_population_adjusted_visitor_home_cbgs', 
          'aggregated_visitor_home_cbgs']})

    median_dwell_cols = [col for col in base.columns if col.endswith('median_dwell')]
    print('Found %d median dwell columns' % len(median_dwell_cols))
    is_null_mat = pd.isnull(base[median_dwell_cols])
    base['num_weeks_with_median_dwell'] = np.sum(~is_null_mat, axis=1)
    base['%s_median_of_median_dwell' % prefix] = np.nanmedian(base[median_dwell_cols].values, axis=1)
    
    cols_to_drop = [col for col in base.columns if '.' in col]  # time-varying are <date>.<col_name>
    base = base.drop(columns=cols_to_drop)
    for k in ['%s_aggregated_cbg_population_adjusted_visitor_home_cbgs' % prefix, 
              '%s_aggregated_visitor_home_cbgs' % prefix]:
        base[k] = base[k].map(lambda x:json.dumps(dict(x))) # cast to json so properly saved in CSV. 
    path_to_csv = os.path.join(NEW_STRATIFIED_BY_AREA_DIR, 'time_aggregated/%s_%s.csv' % (prefix, msa_name))
    print('Saving at', path_to_csv)
    base.to_csv(path_to_csv)
    return base

In [ ]:
stratified_by_msa_dir = os.path.join(NEW_STRATIFIED_BY_AREA_DIR, '20200824_20201019')
fns = os.listdir(stratified_by_msa_dir)
va_msas = []
for fn in fns:
    msa = fn.split('.')[0]  # drop .csv
    states = msa.split('_')[1:]
    if 'VA' in states:
        va_msas.append(msa)
print('Found %d MSAs that overlap with VA' % len(va_msas))
print(va_msas)

In [ ]:
time_period_strings = ['20191230_20200224', '20200302_20200608', '20200615_20200817', '20200824_20201019']
for msa_name in va_msas:
    make_time_aggregated_file_for_msa(msa_name, core_poi_df, pop_df, time_period_strings, dfs=None)

In [ ]:
import pyan
from IPython.display import HTML
HTML(pyan.create_callgraph(filenames="**/*.py", format="html"))


In [ ]:
# process ACS_5YR_2019
PATH_TO_ACS_5YR_BG= r'/media/gpu/easystore/covid_mobility_results/new_census_data/ACS_2019_5YR_BG'
#acs_gdf = gpd.read_file(PATH_TO_ACS_5YR_DATA_GEODATABASE, layer='ACS_2019_5YR_BG')


acs_df = pd.read_csv(os.path.join(PATH_TO_ACS_5YR_BG, 'ACS_2019_5YR_BG.CSV'))

acs_df

In [ ]:
acs_df.head(1)


In [3]:
import pandas as pd
import geopandas as gpd

import os
# calculate area
PATH_TO_ACS_5YR_BG= r'/media/gpu/easystore/covid_mobility_results/new_census_data/ACS_2019_5YR_BG'

acs_gdf = gpd.read_file(os.path.join(PATH_TO_ACS_5YR_BG, 'ACS_2019_5YR_BG ACS_2019_5YR_BG.shp')).to_crs({'proj':'cea'})
acs_gdf.plot()



/media/gpu/easystore/covid-mobility-tool/env_dir/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
acs_df = acs_gdf.drop(columns=['geometry'])
acs_df['Shape_Area'] = acs_gdf['geometry'].area

acs_df.to_csv(os.path.join(PATH_TO_ACS_5YR_BG, 'ACS_2019_5YR_BG.CSV'), index=False)

In [5]:
acs_df_x01 = pd.read_csv(os.path.join(PATH_TO_ACS_5YR_BG, 'ACS_2019_5YR_BG X01_AGE_AND_SEX.csv'))
acs_df_x01


,OBJECTID,GEOID,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,B01001m3,B01001e4,B01001m4,...,B01002He3,B01002Hm3,B01002Ie1,B01002Im1,B01002Ie2,B01002Im2,B01002Ie3,B01002Im3,B01003e1,B01003m1
0,1,15000US010010201001,730,222,293,87,10,12,26,25,...,37.9,21.1,NaN,NaN,NaN,NaN,NaN,NaN,730,222
1,2,15000US010010201002,1263,291,614,148,24,29,29,22,...,38.6,7.5,NaN,NaN,NaN,NaN,NaN,NaN,1263,291
2,3,15000US010010202001,835,222,393,137,40,37,69,63,...,31.9,24.3,65.6,64.9,NaN,NaN,65.6,64.9,835,222
3,4,15000US010010202002,1124,166,665,132,39,25,46,39,...,40.2,6.1,NaN,NaN,NaN,NaN,NaN,NaN,1124,166
4,5,15000US010010203001,2774,330,1423,225,20,24,143,84,...,40.7,12.9,27.8,0.5,27.5,0.2,28.5,0.4,2774,330
5,6,15000US010010203002,733,244,308,133,42,63,0,12,...,33.8,42.7,NaN,NaN,NaN,NaN,NaN,NaN,733,244
6,7,15000US010010204001,761,167,374,94,8,14,20,23,...,54.3,10.4,NaN,NaN,NaN,NaN,NaN,NaN,761,167
7,8,15000US010010204002,1756,251,810,157,23,26,99,63,...,46.4,10.3,18.4,0.9,NaN,NaN,18.4,11.9,1756,251
8,9,15000US010010204003,866,189,496,174,12,20,32,28,...,57.2,6.7,NaN,NaN,NaN,NaN,NaN,NaN,866,189
9,10,15000US010010204004,495,144,269,95,21,34,8,11,...,50.3,24.9,NaN,NaN,NaN,NaN,NaN,NaN,495,144


In [ ]:
acs_df = acs_df.merge(acs_df_x01[[]], )


In [7]:
acs_df.columns

Index(['OBJECTID', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'Shape_Leng', 'Shape_Area', 'GEOID_Data'],
      dtype='object')

In [8]:
acs_df['Shape_Area']


0         2.029247e+06
1         4.292080e+06
2         2.420730e+06
3         1.443893e+06
4         7.353911e+05
5         1.672293e+06
6         2.530616e+06
7         5.275520e+05
8         9.025542e+06
9         7.953756e+06
10        1.328000e+07
11        1.026896e+06
12        1.668555e+06
13        8.840927e+05
14        6.098466e+05
15        5.964309e+05
16        1.226069e+06
17        4.783029e+05
18        6.192645e+05
19        3.654186e+05
20        1.622096e+06
21        2.171500e+06
22        1.373743e+06
23        7.995685e+05
24        2.715685e+06
25        6.747148e+06
26        1.657172e+06
27        3.630738e+08
28        5.349134e+07
29        5.311195e+06
              ...     
220303    1.672826e+05
220304    8.586953e+05
220305    4.067423e+06
220306    3.127694e+05
220307    1.462204e+06
220308    3.983293e+06
220309    6.817024e+06
220310    2.070590e+06
220311    2.230574e+06
220312    5.193712e+05
220313    2.047907e+05
220314    5.739621e+07
220315    1